In [2]:
# Import relevant libraries 

import pandas as pd
import geopandas as gpd
import pysal as ps
import libpysal
import pyproj
import geopy
import cartopy as cp
import geojson
import osmnx as ox
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import bokeh
import folium

# Quest 2: Exploring and manipulating geospatial data with geopandas 

We will cover:
   1. I/O operations 
   2. Getting data from OpenStreetMap (with _OSMnx_)
   3. Reprojecting geospatial data
   4. Basic geoprocessing tools from geopandas (Clip, Spatial join)
    
Note:
- All data is taken from the Geoda Center: https://geodacenter.github.io/. Refer to the main quest page on the StackUp platform for more details.
- Parts of the code marked with the _[quiz]_ tag provide output necessary to answer quiz questions


### 1. Reading/Writing spatial data from/to file
Source: https://geopandas.org/en/stable/docs/user_guide/io.html


__Reading__: geopandas can read almost any vector-based spatial data format (including ESRI shapefile, GeoJSON files and more) and return a GeoDataFrame (gdf) object using the command `geopandas.read_file()`


__Writing__: GeoDataFrames can be exported to many different standard formats using `geopandas.GeoDataFrame.to_file()`

#### Read dataset containing health and socioeconomics statistics in Chicago (ComArea_ACS14_f.shp) from your working directory:
`your_var_name = gpd.read_file("your filepath")`

In [ ]:
# your code here

#### Inspect your gdf:
`your_var_name.head()`


Scroll through the columns in your gdf. The last column contains the geometry attribute.

In [ ]:
# your code here

#### Inspect the 'geometry' attribute column:
`your_var_name['geometry'].head()`


*[quiz]* Take note of the data type (dtype).

In [ ]:
# your code here

#### Display the geometry of the fourth feature (i.e. index = 3) in the gdf:
`your_var_name['geometry'][3]`


*[quiz]* What shape best describes the shape of this feature?

In [ ]:
# your code here

#### Display the other attributes of the same feature (i.e. index = 3):
`your_var_name.iloc[3]`


Take note of the _community_ name of this feature. You will need this for the next step.

In [ ]:
# your code here

### 2. Fetching data from OpenStreetMap
For more details, refer to: https://osmnx.readthedocs.io/en/stable/internals.html#osmnx-geocoder-module

We can use the osmnx package to read data from OpenStreetMap to GeoDataFrames. Now, we will fetch the city boundary of Chicago using `osmnx.geocode_to_gdf(query)`

You can save your query (i.e. name of the country/city/region that you are interested in) as a separate variable. For example, if we want to fetch the city boundary of Chicago:

In [ ]:
city_name = 'Chicago, Illinois'
city = ox.geocoder.geocode_to_gdf(city_name, which_result=1)

#### Now, do the same to fetch the boundary of the community you identified above:

In [23]:
community_name = 'XXX, Chicago' # replace 'XXX' with the actual community name from above
community = # your code here

#### Inspect your new community variable:

# ⓘ 
> This community area is part of the larger _Bronzeville_ area, an important center for African-American culture and businesses in Chicago. Bronzeville is also one of 10 priority communities selected as a part of the initial phase of a community development initiative by its Mayor.

In [ ]:
community

#### and display its geometry:

The shape you see should be more or less the same as the one you saw earlier (from the gdf)!

In [ ]:
community['geometry'][0]

#### Apart from boundaries, you can also get other map features from OSM such as roads, building footprints, waterways etc.

See https://wiki.openstreetmap.org/wiki/Map_features for more information.

#### Now, let us try to fetch the building footprint of all residential buildings within Chicago city boundary:

We will include `'apartments', 'terrace', 'house', 'detached', 'semidetached_house'` as the building tags.

In [88]:
residential_chicago = ox.geometries.geometries_from_polygon(city['geometry'][0], tags = {'building': ['apartments', 'terrace', 'house', 'detached', 'semidetached_house']})

#### Inspect the 'residential_chicago' gdf:

In [ ]:
residential_chicago.head()

### 3. Projections and coordinate reference systems (CRS)
For more details, refer to: https://geopandas.org/en/stable/docs/user_guide/projections.html

When reading geospatial files with `geopandas.read_file()`, things should mostly work out of the box with a proper CRS defined. You can use `GeoDataFrame.crs` to display CRS information:

In [ ]:
your_var_name.crs # replace your_var_name with the name of your chicago statistics gdf accordingly

*Note: Sometimes, you may need to manually set a CRS or perform a reprojection if the CRS information is not as expected.*

__Reprojections__: Re-projecting is the process of changing the representation of locations from one coordinate system to another. All projections of locations on the Earth into a two-dimensional plane are distortions, the projection that is best for your application may be different from the projection associated with the data you import. In these cases, data can be re-projected using `GeoDataFrame.to_crs()`. Specify the name of the CRS by their _EPSG_ code. If you are working with multiple geospatial data files, all must share the same CRS in order for the data to be mapped accurately.

#### Geographic CRS vs. projected CRS
It is generally preferred to work with projected CRS than geographic CRS. Also, computing geometric properties (e.g., area) in a geographic CRS will likely to give inaccurate results. We usually re-project geometries to a projected CRS before performing such operations.

#### Reproject your gdf to one of the most commonly used metric projected CRS (EPSG:26916) for Chicago:

`your_var_name = your_var_name.to_crs("EPSG:26916")`

`your_var_name.crs`

In [ ]:
# your code here

#### Now do the same for the _community_ variable and reproject it to the same _EPSG:26916_ CRS:

In [ ]:
community = community.to_crs("XXXX") # replace XXXX with the appropropriate CRS
community.crs

#### *[quiz]* Let's now see how big the GB neighbourhood is:

_Refer to the [geopandas documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.area.html#) and complete the code below to compute the area of the GB neighbourhood._

In [ ]:
# your code here to compute the area of GB

#### We can also compute the perimeter of the neighbourhood with:

In [ ]:
community.length

__Note:__ The units of follow those defined by the CRS used. In this case (EPSG:26916), the area is expressed in _square metres_, and perimeter in _metres_.

### 4. Basic tools from geopandas
For more details, refer to: https://geopandas.org/en/stable/docs/reference/tools.html

In this section, we will cover two main tools:
- Clip
- Spatial join

#### Clip (Source: https://gisgeography.com/clip-tool-gis/)

The Clip Tool cuts out an input layer to a defined feature boundary. The clipping layer must be a polygon. But the input layer can be points, lines, or polygons.

In the image below, the first figure represents the input layer and the second figure represents the clipping layer.

![](https://gisgeography.com/wp-content/uploads/2020/10/Clip-Tool-1536x487.png)

We can use `input_layer.clip(clipping_layer)` to perform the Clip. We will now look at supermarket locations in Chicago and to perform a Clip. You can also refer to this other example in the geopandas documentation: https://geopandas.org/en/stable/gallery/plot_clip.html

#### Load and inspect Chicago supermarkets data:

In [ ]:
supermarkets = gpd.read_file("your file path") # replace file path to 'chicago_sup.shp' in your working directory
supermarkets.head()

*[Quiz]* What type of geometry do we see in this supermarket dataset?

#### Re-project supermarkets to EPSG:26916:

In [113]:
supermarkets = supermarkets.to_crs("EPSG:26916")

#### Plot Chicago neighbourhoods (from Chicago statistics gdf) and supermarkets:

_Please refer to the [geopandas documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html) to complete the code below._

In [ ]:
fig, (ax1) = plt.subplots(figsize=(5, 5))
your_var_name.plot(ax=ax1) # replace your_var_name (Chicago statistics gdf from Part 1) accordingly

# Now, write your code here to add supermarkets to the plot.
# You can use the 'color' parameter to make the supermarkets stand out.

#### Get boundary of Lincoln Park, one of Chicago's largest neighbourhoods, from OpenStreetMap:

# ⓘ 
> Lincoln Park is one of the most attractive (and expensive!) neighbourhoods in Chicago thanks to its proximity to the city center, ample green space, and vibrant atmosphere. 

In [ ]:
lp_chicago = ox.geocoder.geocode_to_gdf('Lincoln Park, Chicago', which_result=2)
lp_chicago 

Check that _'class' = boundary_ and _'type' = administrative_. Otherwise, play around with the `which_result` parameter.

#### Re-project lp_chicago to EPSG:26916:

In [127]:
lp_chicago = lp_chicago.to_crs("EPSG:26916")

#### Visualise the Lincoln Park neighbourhood on the map:

_Refer to the geopandas documentation on [mapping and plotting tools](https://geopandas.org/en/stable/docs/user_guide/mapping.html) and [boundaries](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.boundary.html) to plot a **red boundary** of the Lincoln Park neighbourhood._ 

In [ ]:
fig, (ax1) = plt.subplots(figsize=(5, 5))
your_var_name.plot(ax=ax1) # replace your_var_name (Chicago statistics gdf from Part 1) accordingly
supermarkets.plot(# complete the code here to add supermarkets (color = 'black') to the map)
    
# Now, write your code here to add a red boundary of the Lincoln Park neighbourhood to the plot.  

#### We will now clip the supermarket dataset using the Lincoln Park neighbourhood boundary:

In [ ]:
supermarkets_lp = supermarkets.clip(lp_chicago) # ignore warning

#### *[Screenshot]* Plot the results of the Clip:

_Complete the lines of code below accordingly._

In [ ]:
fig, (ax1) = plt.subplots(figsize=(5, 5))
your_var_name.plot(ax=ax1) # replace your_var_name (Chicago statistics gdf from Part 1) accordingly
# Add your code here to plot (in black) the supermarkets IN Lincoln Park, i.e., the clipped supermarket dataset
# Add your code here to plot (in red) the boundary of Lincoln Park

*[Quiz]* How many supermarkets are located in Lincoln Park, Chicago?

# ⓘ 
> Access to essential services and amenities is an important issue that urban planners have to consider in their work. By mapping their locations (e.g. supermarkets as we have just done) in relation to other key features of the cities like the road network and building footprint, planners can identify areas in which access to such services are lacking. 

#### Spatial Join (Source: https://gisgeography.com/spatial-join/)

The spatial join tool inserts the columns from one feature table to another based on location or proximity. Consider the _supermarkets_ dataset that we have just seen. By running a spatial join, you can transfer the information about the supermarkets to the Chicago gdf that we have been working with. As long as the supermarkets dataset contains location information, any field can be joined to the neighbourhoods listed in the Chicago health and socioeconomic statistics dataset.

In geopandas, we can perform a spatial join with 

`geopandas.sjoin(left_df, right_df, how='inner', predicate='intersects')`

There are different __predicates based on the nature of the spatial relationships__ (see https://gisgeography.com/spatial-join/ for graphical representations of each) that can be used to define the join:
- __Intersect__: Two features touch at any location. (default)
- __Contains/Completely within__: The join feature is completely contained within the target feature. 
- __Identical__: Both features match identically. When two features are exactly without having any difference or variant of any kind, the features are identical.
- __Closest/Nearest__: The join feature is nearest/closest to the target feature. When there are several possible features to join, this spatial join takes only the closest feature.
- __Within a distance__: Two features are within a set distance. This spatial relationship is defined by how far two or more entities can be found on a map.

There are also different __types of joins (how)__ (see also https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html#geopandas.sjoin) to determine how the columns from one table should be joined to the other:
- __inner__: use intersection of keys from both dfs; retain only left_df geometry column
- __left__: use keys from left_df; retain only left_df geometry column
- __right__: use keys from right_df; retain only right_df geometry column


We will now add some supermarket information to the various neighbourhoods in Chicago. You can refer to another example here: https://geopandas.org/en/stable/gallery/spatial_joins.html.

#### Re-inspect the Chicago statistics gdf (your_var_name):

In [ ]:
your_var_name.head() # replace your_var_name accordingly

We can see that there are __87__ columns in the Chicago statistics dataset.

#### Re-inspect the supermarket gdf:

In [ ]:
supermarkets.head()

We can see that there are __8__ columns in the supermarkets dataset.

#### Now, we will perform the spatial join with:

`left_df = your_var_name`

`right_df = supermarkets`

According to the information provided above on spatial relationships and join types, complete the code below `how` and `predicate` parameters to perform a spatial join where

    - the keys of the left_df are used and only the geometry of the left_df is retained
    - features are joined as long as they touch at any location

In [ ]:
joined_df = gpd.sjoin(your_var_name, supermarkets, how=, predicate=) # replace your_var_name accordingly and complete the code
joined_df.head()

Scroll across all the columns to the end and we can see that 7 columns from the _supermarkets_ dataset (without geometry) have been joined to the Chicago health and socioeconomic statistics dataset along with an index column ('index_right'). Note that the rows with __NaN__ indicate neighbourhoods that did not have any supermarkets (0 points intersecting the polygon).

Also note that results will include multiple output records for a single input record with multiple intersecting features. This means that a neighbourhood (e.g. Lincoln Park) with three supermarkets as we have seen earlier will appear three times in the join output:

In [ ]:
joined_df.loc[joined_df['community'] == "LINCOLN PARK"]

#### Now, we wanted to find the percentage of Chicago neighbourhoods with at least one supermarket, we can do the following:

1: Establish a df giving us the neighbourhoods in which each supermarket (from the _supermarkets_ dataset) is located.

In [179]:
nbhds_w_supermarkets = joined_df[pd.notna(joined_df.Chain)]

2: Count the number of unique neighbourhoods ('community' column) with at least one supermarket.

In [180]:
nbhds_w_supermarkets = nbhds_w_supermarkets['community'].nunique()

3: *[quiz]* Compute percentage:

In [ ]:
round(nbhds_w_supermarkets/len(your_var_name)*100,0) # replace your_var_name accordingly (Chicago statistics from Part 1)

### Well done!

We’ve come to the end of this quest where you:

   1. Performed I/O operations 
   2. Retrieved data from OpenStreetMap
   3. Reprojected geospatial data
   4. Used basic geoprocessing tools from geopandas
   5. Created basic plots of spatial data

Now go back to the StackUp website to prepare your deliverables for submission!